In [1]:
#import libraries
from random_word import RandomWords
from PIL import Image, ImageDraw
import pytesseract
import cv2

In [2]:
#Reference 1: Create images with Python PIL and Pillow and write text on them 
#<https://code-maven.com/create-images-with-python-pil-pillow>

#Reference 2: random-word 1.0.3
#<https://pypi.org/project/random-word/>

def random_word_image_generator():
    r = RandomWords()

    #Create a new gray image
    img = Image.new('RGB', (200, 70), color = (200, 200, 200))

    #Return a single random word
    word = r.get_random_word(hasDictionaryDef="true", minLength=8, maxLength=12)

    #Draw the random word on the gray image created
    d = ImageDraw.Draw(img)
    d.text((20,10), word, fill=(0,0,0))

    #Save the image
    img.save('word.jpg')
    
    print(word)
    
#Run the random word image generator function
random_word_image_generator()

crackable


In [3]:
#Reference 3: Detect text region in image using Opencv 
#<https://stackoverflow.com/questions/24385714/detect-text-region-in-image-using-opencv>

def text_region_detect(file_name):
    
    #Load original image for processing
    img = cv2.imread(file_name)
    
    #Convert the original image to grayscale
    img2gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #Classify the pixel values to create a binary (black and white) image
    _, new_img = cv2.threshold(img2gray, 127, 255, cv2.THRESH_BINARY_INV)
     
    #Dilate image to expand white pixels into one another
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    dilated = cv2.dilate(new_img, kernel, iterations=5)
    
    #Create contour from dilated image
    _, contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    for contour in contours:
        #Get rectangle bounding contour
        [x, y, h, w] = cv2.boundingRect(contour)

    #Load original image again 
    img_final = cv2.imread(file_name)
        
    #Define the coordinates and dimensions of contour on original image
    cropped = img_final[y :y +  w , x : x + h]
        
    #Classify the pixel values to create a binary (black and white) image
    _ , cropped = cv2.threshold(cropped, 127, 255, cv2.THRESH_BINARY)
        
    #Enlarge image to improve accuracy of OCR
    (NewHeight, NewWidth) = (h * 4, w * 4)
    cropped = cv2.resize(cropped, (NewHeight, NewWidth))
        
    #Classify the pixel values to create a binary (black and white) image
    _ , cropped = cv2.threshold(cropped, 127, 255, cv2.THRESH_BINARY)

    #save contour as new image for OCR
    cv2.imwrite('cropped.jpg', cropped)
   
    #Draw rectangle around contour on original image
    cv2.rectangle(img, (x, y), (x + h, y + w), (255, 0, 255), 2)
    cv2.putText(img,"This is the cropped area", (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 255))
    cv2.putText(img,"Press any key to continue", (20,60), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 0, 255))
        
    #Write original image with added contours
    cv2.imshow('text_region_detect_result', img)
    cv2.waitKey(0)
    #Press any key to continue the program

#Run the text region detection function
file_name = 'word.jpg'

text_region_detect(file_name)

In [4]:
#Destroy the imshow window
cv2.destroyAllWindows()

#Recognize the text using Tesseract OCR with Neural nets LSTM engine and English configurations
print(pytesseract.image_to_string(Image.open('cropped.jpg'), 
                                  config = ('-l eng --oem 1')))


crackable
